Системы аналитических вычислений.

Студент: Пермяков Никита Александрович, М8О-208Б-19

Лабораторная работа № 2

### Вариант А. дихотомия и метод Ньютона

Поиск решения осуществляется путём построения последовательных приближений и основан на принципах простой итерации. Метод обладает квадратичной сходимостью. 

In [51]:
# Дано
f(x) = x^3 - 9*x^2 + 23*x - 15
a = (0, 2, 0.5)
b = (0, 2, 0.5)
step = (1, 2, 1)

In [52]:
# Решим уравнение
solve([f(x) == 0], x)

[x == 1, x == 5, x == 3]

In [53]:
# Метод дихотомии - ищем точку пересечения с Ox
def dichotomy(funс, a, b, eps):
    if funс(a) * funс(b) > 0:
        return None
    m = (a + b) / 2.
    if abs(funс(m).n()) < eps:
        return m
    if func(m) * funс(a) < 0:
        return dichotomy(funс, a, m)
    else:
        return dichotomy(funс, m, b)

In [60]:
# Метод Ньютона - ищем точку пересечения с Ox
def newton(fun, a, b, eps):
    x1 = a
    x2 = (a + b) / 2
    df = fun.derivative()
    while abs(x2 - x1) >= eps:
        x1 = x2
        x2 = x1 - fun(x=x1) / df(x=x1)
    if abs(fun(x=x1)) <= 1e-3:
        return x1
    return None

In [61]:
@interact
def func_interact(a=a, b=b, step=step):
    xmin = -1
    xmax = 5.5
    ymin = -4
    ymax = 5
    eps = 1e-5
    
    #функция
    f(x) = x^3 - 9*x^2 + 23*x - 15
    #значения корней методом дихотомии и Ньютона
    di = [dichotomy(f, a, b, eps)]
    di.append(dichotomy(f, a+step, b+step, eps))
    di.append(dichotomy(f, a+2*step, b+2*step, eps))
    newt = [newton(f, a, b, eps)]
    newt.append(newton(f, a+step, b+step, eps))
    newt.append(newton(f, a+2*step, b+2*step, eps))
    show("метод дихотомии - красные графики")
    show("метод Ньютона - синие")
    gr = [plot(f, (x, xmin, xmax), ymin=ymin, ymax=ymax, aspect_ratio=1, figsize=[3,2], color='red')]*3
    gr.extend([plot(f, (x, xmin, xmax), ymin=ymin, ymax=ymax, aspect_ratio=1, figsize=[3,2], color='blue')]*3)
    for i in range(len(gr)):
        if i <= 2 and di[i] is not None:
            gr[i] += point((float(di[i]), 0), color='green', aspect_ratio=1, size=30)
        if i > 2 and newt[i-3] is not None:
            gr[i] += point((float(newt[i-3]), 0), color='green', aspect_ratio=1, size=30)

    #вывод массива графиков
    graphics_array([[gr[0], gr[1], gr[2]], [gr[3], gr[4], gr[5]]]).show(frame=True)

    str_ = "Корни, полученные методом дихотомии: "
    for d in di:
        if d is not None:
            str_ += str(round(float(d))) + " "
    str_ += "\nКорни, полученные методом Ньютона: "
    for n in newt:
        if n is not None:
            str_ += str(round(float(n))) + " "
    show(str_)

Interactive function <function func_interact at 0x7fd54c9a44c0> with 3 widgets
  a: FloatSlider(value=1.0, min…